# eWRC API

Simple Python API for eWRC results.

In [39]:
import pandas as pd
import re
from dakar_utils import getTime

In [40]:
import requests
import lxml.html as LH
from bs4 import BeautifulSoup

## Generic Utilities

Utility functions.

In [41]:
def soupify(url):
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'lxml') # Parse the HTML as a string
    return soup

In [42]:
def dfify(table):
    df = pd.read_html('<html><body>{}</body></html>'.format(table))[0]
    df = df.dropna(axis=1, how='all').dropna(axis=0, how='all')
    return df

In [43]:
import unicodedata

In [44]:
def cleanString(s):
    s = unicodedata.normalize("NFKD", str(s))
    #replace multiple whitespace with single space
    s = ' '.join(s.split())
    
    return s

## Timing Utilities

In [45]:
def diffgapsplitter(col):
    #Normalise
    col=col.fillna('+0+0')
    #Remove leading +
    col=col.str.strip('+')
    #Split...
    col = col.str.split('+',expand=True)
    #Rename columns
    col = col.rename(columns={0:'Gap', 1:'Diff'})
    #Convert to numerics
    col['Gap'] = col['Gap'].apply(getTime)#.astype(float)
    col['Diff'] = col['Diff'].apply(getTime)
    return col

## Scraping Functions

In [46]:
base_url = 'https://www.ewrc-results.com'

In [47]:
def get_stage_result_links(stub):
    #If navigation remains constant, items are in third list
    
    rally_stage_results_url='https://www.ewrc-results.com/results/{stub}/'.format(stub=stub)
    
    links={}

    soup = soupify(rally_stage_results_url)
    for li in soup.find_all('ul')[2].find_all('li'):
        #if 'class' in li.attrs:
        #    print(li['class'])
        #A class is set for service but not other things
        if 'class' not in li.attrs:
            a = li.find('a')
            if 'href' in a.attrs:
                #links.append(a['href'])
                links[f'SS{a.text}'] = a['href']
                
    return links

In [48]:
#url='https://www.ewrc-results.com/results/54762-corbeau-seats-rally-tendring-clacton-2019/'
rally_stub = '54762-corbeau-seats-rally-tendring-clacton-2019'
tmp = get_stage_result_links(rally_stub)
tmp

{'SS1': '/results/54762-corbeau-seats-rally-tendring-clacton-2019/?s=230570',
 'SS2': '/results/54762-corbeau-seats-rally-tendring-clacton-2019/?s=230571',
 'SS3': '/results/54762-corbeau-seats-rally-tendring-clacton-2019/?s=230572',
 'SS4': '/results/54762-corbeau-seats-rally-tendring-clacton-2019/?s=230573',
 'SS5': '/results/54762-corbeau-seats-rally-tendring-clacton-2019/?s=230574',
 'SS6': '/results/54762-corbeau-seats-rally-tendring-clacton-2019/?s=230575',
 'SS7': '/results/54762-corbeau-seats-rally-tendring-clacton-2019/?s=230576',
 'SS8': '/results/54762-corbeau-seats-rally-tendring-clacton-2019/?s=230577',
 'SS9': '/results/54762-corbeau-seats-rally-tendring-clacton-2019/?s=230578',
 'SS10': '/results/54762-corbeau-seats-rally-tendring-clacton-2019/?s=230579',
 'SS11': '/results/54762-corbeau-seats-rally-tendring-clacton-2019/?s=230580',
 'SS12': '/results/54762-corbeau-seats-rally-tendring-clacton-2019/?s=230581'}

In [49]:
stage_result_cols = ['Pos', 'CarNum', 'Desc', 'Class', 'Time', 'GapDiff', 'Speedkm', 'Stage',
       'StageName', 'StageDist', 'Gap', 'Diff', 'Speed', 'Dist', 'entryId',
       'model', 'navigator', 'PosNum']

In [50]:
stage_overall_cols = ['PosChange', 'CarNum', 'Desc', 'Class', 'Time', 'GapDiff', 'Speedkm',
       'Stage', 'StageName', 'StageDist', 'Pos', 'Change', 'Gap', 'Diff',
       'Speed', 'Dist']

In [51]:
retirement_cols = ['CarNum', 'driverNav', 'Model', 'Status']
retirement_extra_cols = ['Driver', 'CoDriver', 'Stage']

In [52]:
penalty_cols = ['CarNum', 'driverNav', 'Model', 'PenReason']
penalty_extra_cols = ['Driver', 'CoDriver', 'Stage', 'Time','Reason']

In [53]:
from numpy import nan

In [54]:
from parse import parse   

In [55]:
def get_stage_results(stub):
    soup = soupify('{}{}'.format(base_url, stub))
    
    pattern = 'SS{stage} {name} - {dist:f} km - {datetime}'
    details = soup.find('h4').text
    parse_result = parse(pattern, details)

    stage_num = f"SS{parse_result['stage']}"
    stage_name = parse_result['name']
    stage_dist =  parse_result['dist']
    stage_datetime = parse_result['datetime']

    tables = soup.find_all('table')
    stage_result = tables[0]
    stage_overall = tables[1]
    
    
    # Stage Result
    df_stage_result = dfify(stage_result)
    df_stage_result.columns=['Pos','CarNum','Desc','Class', 'Time','GapDiff', 'Speedkm']
    
    df_stage_result['Stage'] = stage_num
    df_stage_result['StageName'] = stage_name
    df_stage_result['StageDist'] = stage_dist
    
    df_stage_result['GapDiff'].fillna('+0+0').str.strip('+').str.split('+',expand=True).rename(columns={0:'Gap', 1:'Diff'})
    df_stage_result[['Gap','Diff']] = diffgapsplitter(df_stage_result['GapDiff'])
    df_stage_result[['Speed','Dist']] = df_stage_result['Speedkm'].str.extract(r'(?P<Speed>[^.]*\.[\d])(?P<Dist>.*)')
    
    rows=[]
    for d in stage_result.findAll("td", {"class": "stage-results-drivers"}):
        entryId = d.find('a')['href']
        #print(str(d)) #This gives us the raw HTML in the soup element
        driverNav = d.find('a').text.split('-')
        model=d.find('a').nextSibling.nextSibling
        rows.append( {'entryId':entryId,
                       'model':model,
                      'driver':cleanString(driverNav[0]),
                      'navigator':cleanString(driverNav[1])}) 

    df_stage_result[['driver','entryId','model','navigator']] = pd.DataFrame(rows)
    #Should we cast the Pos to a numeric too? Set = to na then ffill down?
    df_stage_result['PosNum'] = df_stage_result['Pos'].replace('=',nan).astype(float).fillna(method='ffill').astype(int)
    df_stage_result.set_index('driver',drop=True, inplace=True)
    
    # Stage Overall
    df_stage_overall = dfify(stage_overall)

    cols = ['PosChange', 'CarNum', 'Desc','Class', 'Time', 'GapDiff', 'Speedkm' ]
    df_stage_overall.columns = cols

    df_stage_overall['Stage'] = stage_num
    df_stage_overall['StageName'] = stage_name
    df_stage_overall['StageDist'] = stage_dist
    
    df_stage_overall[['Pos','Change']] = df_stage_overall['PosChange'].astype(str).str.extract(r'(?P<Pos>[\d]*)\.\s?(?P<Change>.*)?')
    df_stage_overall['GapDiff'].fillna('+0+0').str.strip('+').str.split('+',expand=True).rename(columns={0:'Gap', 1:'Diff'})
    df_stage_overall[['Gap','Diff']] = diffgapsplitter(df_stage_overall['GapDiff'])
    df_stage_overall[['Speed','Dist']] = df_stage_overall['Speedkm'].str.extract(r'(?P<Speed>[^.]*\.[\d])(?P<Dist>.*)')

    
    # Retirements
    df_stage_retirements = pd.DataFrame(columns=retirement_cols+retirement_extra_cols)
    retired = soup.find('div',{'class':'retired-inc'})
    if retired:
        df_stage_retirements = dfify(retired.find('table'))
        df_stage_retirements.columns = retirement_cols
        df_stage_retirements[['Driver','CoDriver']] = df_stage_retirements['driverNav'].str.extract(r'(?P<Driver>.*)\s+-\s+(?P<CoDriver>.*)')
        df_stage_retirements['Stage'] = stage_num
    
    # Penalties
    df_stage_penalties = pd.DataFrame(columns=penalty_cols+penalty_extra_cols)

    penalty = soup.find('div',{'class':'penalty-inc'})
    if penalty:
        df_stage_penalties = dfify(penalty.find('table'))
        df_stage_penalties.columns = penalty_cols
        df_stage_penalties[['Driver','CoDriver']] = df_stage_penalties['driverNav'].str.extract(r'(?P<Driver>.*)\s+-\s+(?P<CoDriver>.*)')
        df_stage_penalties[['Time','Reason']] = df_stage_penalties['PenReason'].str.extract(r'(?P<Time>[^\s]*)\s+(?P<Reason>.*)')
        df_stage_penalties['Stage'] = stage_num
    
    return df_stage_result, df_stage_overall, df_stage_retirements, df_stage_penalties

In [56]:
partial_stub = '/results/54762-corbeau-seats-rally-tendring-clacton-2019/'
partial_stub='/results/42870-rallye-automobile-de-monte-carlo-2018/'
#stub = tmp['SS3']
stage_result, stage_overall, stage_retirements, stage_penalties = get_stage_results(partial_stub)

In [57]:
stage_result.head()

,Pos,CarNum,Desc,Class,Time,GapDiff,Speedkm,Stage,StageName,StageDist,Gap,Diff,Speed,Dist,entryId,model,navigator,PosNum
driver,,,,,,,,,,,,,,,,,,
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642095/,1.0,10,Meeke Kris - Nagle PaulCitroën C3 WRC,RC1M,10:06.7,NaN,80.6,SS17,La Cabanette - Col de Braus 2 [Power Stage],13.58,0.0,0.0,80.6,,Citroën C3 WRC,Meeke Kris,Nagle Paul,1
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642081/,2.0,5,Neuville Thierry - Gilsoul N.Hyundai i20 Coupe...,RC1M,10:09.0,+2.3+2.3,80.30.17,SS17,La Cabanette - Col de Braus 2 [Power Stage],13.58,2.3,2.3,80.3,0.17,Hyundai i20 Coupe WRC,Neuville Thierry,Gilsoul N.,2
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642085/,3.0,4,Mikkelsen Andreas - Jæger A.Hyundai i20 Coupe WRC,RC1M,10:11.1,+4.4+2.1,80.00.32,SS17,La Cabanette - Col de Braus 2 [Power Stage],13.58,4.4,2.1,80.0,0.32,Hyundai i20 Coupe WRC,Mikkelsen Andreas,Jæger A.,3
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642089/,4.0,7,Latvala Jari-Matti - Anttila M.Toyota Yaris WRC,RC1M,10:13.6,+6.9+2.5,79.70.51,SS17,La Cabanette - Col de Braus 2 [Power Stage],13.58,6.9,2.5,79.7,0.51,Toyota Yaris WRC,Latvala Jari,Matti,4
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642063/,5.0,1,Ogier Sébastien - Ingrassia J.Ford Fiesta WRC,RC1M,10:14.8,+8.1+1.2,79.50.60,SS17,La Cabanette - Col de Braus 2 [Power Stage],13.58,8.1,1.2,79.5,0.60,Ford Fiesta WRC,Ogier Sébastien,Ingrassia J.,5


In [58]:
stage_overall

,PosChange,CarNum,Desc,Class,Time,GapDiff,Speedkm,Stage,StageName,StageDist,Pos,Change,Gap,Diff,Speed,Dist
0,1.,1,Ogier S. - Ingrassia J.Ford Fiesta WRC,RC1M,4:18:55.5,NaN,90.0,SS17,La Cabanette - Col de Braus 2 [Power Stage],13.58,1,,0.0,0.0,90.0,
1,2.,8,Tänak Ott - Järveoja M.Toyota Yaris WRC,RC1M,4:19:53.8,+58.3+58.3,89.70.15,SS17,La Cabanette - Col de Braus 2 [Power Stage],13.58,2,,58.3,58.3,89.7,0.15
2,3.,7,Latvala J. - Anttila M.Toyota Yaris WRC,RC1M,4:20:47.5,+1:52.0+53.7,89.40.29,SS17,La Cabanette - Col de Braus 2 [Power Stage],13.58,3,,112.0,53.7,89.4,0.29
3,4. +1,10,Meeke Kris - Nagle PaulCitroën C3 WRC,RC1M,4:23:38.6,+4:43.1+2:51.1,88.40.73,SS17,La Cabanette - Col de Braus 2 [Power Stage],13.58,4,+1,283.1,171.1,88.4,0.73
4,5. +2,5,Neuville T. - Gilsoul N.Hyundai i20 Coupe WRC,RC1M,4:23:49.3,+4:53.8+10.7,88.40.76,SS17,La Cabanette - Col de Braus 2 [Power Stage],13.58,5,+2,293.8,10.7,88.4,0.76
5,6.,2,Evans Elfyn - Barritt D.Ford Fiesta WRC,RC1M,4:23:50.3,+4:54.8+1.0,88.40.76,SS17,La Cabanette - Col de Braus 2 [Power Stage],13.58,6,,294.8,1.0,88.4,0.76
6,7. −3,9,Lappi Esapekka - Ferm JanneToyota Yaris WRC,RC1M,4:23:53.0,+4:57.5+2.7,88.40.77,SS17,La Cabanette - Col de Braus 2 [Power Stage],13.58,7,−3,297.5,2.7,88.4,0.77
7,8.,3,Bouffier Bryan - Panseri X.Ford Fiesta WRC,RC1M,4:26:35.0,+7:39.5+2:42.0,87.51.18,SS17,La Cabanette - Col de Braus 2 [Power Stage],13.58,8,,459.5,162.0,87.5,1.18
8,9.,11,Breen Craig - Martin ScottCitroën C3 WRC,RC1M,4:28:02.20:10,+9:06.7+1:27.2,87.01.41,SS17,La Cabanette - Col de Braus 2 [Power Stage],13.58,9,,546.7,87.2,87.0,1.41
9,10.,32,Kopecký Jan - Dresler P.Škoda Fabia R5,RC2,4:35:38.5,+16:43.0+7:36.3,84.62.58,SS17,La Cabanette - Col de Braus 2 [Power Stage],13.58,10,,1003.0,456.3,84.6,2.58


In [59]:
stage_retirements

,CarNum,driverNav,Model,Status,Driver,CoDriver,Stage


In [60]:
stage_penalties

,CarNum,driverNav,Model,PenReason,Driver,CoDriver,Time,Reason,Stage
0,35,de Mevius G. - Louka L.,Peugeot 208 T16,19:00 Stewards decision,de Mevius G.,Louka L.,19:00,Stewards decision,SS17


In [61]:
from parse import parse

In [96]:
def get_stage_times(stub):
    url='https://www.ewrc-results.com/times/{stub}/'.format(stub=stub)

    soup = soupify(url)
    
    times = soup.find('div',{'class':'times'}).findChildren('div' , recursive=False)

    #The rows are essentially grouped in twos after the header row
    cols = [c.text for c in times[0].findAll('div')]
    
    groupsize=2
    groups = [times[i:i+groupsize] for i in range(1, len(times), groupsize)]
    
    NAME_SUBGROUP = 0
    TIME_SUBGROUP = 1
    
    carNumMatch = lambda txt: re.search('#(?P<carNum>[\d]*)', cleanString(txt))
    carModelMatch = lambda txt:  re.search('</a>\s*(?P<carModel>.*)</div>', cleanString(txt))
    
    pattern = '''<div class="times-one-time">{stagetime}<br/><span class="times-after">{overalltime}</span><br/>{pos}</div>'''
    
    t=[]
    i=0

    penaltypattern='class="r7_bold_red">{penalty}</span>'

    for g in groups:
        i=i+1
        driverNav_el = g[NAME_SUBGROUP].find('a')
        driverNav = driverNav_el.text
        driver,navigator = driverNav.split(' - ')
        entryId = driverNav_el['href']
        retired = '<span class="r8_bold_red">R</span>' in str(g[NAME_SUBGROUP])
        carNum = carNumMatch(g[NAME_SUBGROUP]).group('carNum')
        carModel = carModelMatch(g[NAME_SUBGROUP]).group('carModel')
        classification = pd.to_numeric(g[NAME_SUBGROUP].find('span').text.replace('R','').strip('').strip('.'))

        stagetimes = []
        overalltimes = []
        penalties=[]
        positions = []

        for stages in g[TIME_SUBGROUP].findAll('div'):
            txt = cleanString(stages)
            stagetimes_data = parse(pattern, txt )
            if stagetimes_data:
                stagetimes.append(stagetimes_data['stagetime'])
                overalltimes.append(stagetimes_data['overalltime'])

                #Need to parse this
                #There may be penalties in the pos
                penalty = 0
                p = stagetimes_data['pos'].split()
                if p[-1].endswith('</span>'):
                    penalty = parse(penaltypattern, p[-1] )
                    if penalty:
                        #This really needs parsing into a time; currently of form eg 0:10
                        penalty = penalty['penalty']
                    p = int(p[-2].split('.')[0])
                else:
                    p = int(p[-1].strip('.'))
                positions.append(p)
                penalties.append(penalty)

        t.append({'entryId':entryId,
                  'driverNav': driverNav,
                  'driver':driver.strip(),
                 'navigator':navigator.strip(),
                  'carNum':carNum,
                  'carModel':carModel,
                  'retired':retired,
                  'Pos': classification,
                 'stagetimes':stagetimes,
                 'overalltimes':overalltimes,
                 'positions':positions, 'penalties':penalties})


    df_allInOne = pd.DataFrame(t).set_index(['entryId'])
    
    df_overall = pd.DataFrame(df_allInOne['overalltimes'].tolist(), index= df_allInOne.index)
    df_overall.columns = range(1,df_overall.shape[1]+1)
    
    df_overall_pos = pd.DataFrame(df_allInOne['positions'].tolist(), index= df_allInOne.index)
    df_overall_pos.columns = range(1,df_overall_pos.shape[1]+1)

    df_stages = pd.DataFrame(df_allInOne['stagetimes'].tolist(), index= df_allInOne.index)
    df_stages.columns = range(1,df_stages.shape[1]+1)
    
    df_stages_pos = df_stages.rank(method='min')
    df_stages_pos.columns = range(1,df_stages_pos.shape[1]+1)
    
    xcols = df_overall.columns

    for ss in xcols:
        df_overall[ss] = df_overall[ss].apply(getTime)
        df_stages[ss] = df_stages[ss].apply(getTime)

    return df_allInOne, df_overall, df_stages, df_overall_pos, df_stages_pos

In [63]:
url='https://www.ewrc-results.com/times/54762-corbeau-seats-rally-tendring-clacton-2019/'
#url='https://www.ewrc-results.com/times/42870-rallye-automobile-de-monte-carlo-2018/'

In [64]:
rally_stub = '42870-rallye-automobile-de-monte-carlo-2018'
df_allInOne, df_overall, df_stages, \
    df_overall_pos, df_stages_pos = get_stage_times(rally_stub)

In [65]:
display(df_allInOne)
display(df_overall)
display(df_stages)
display(df_overall_pos)

,driverNav,driver,navigator,carNum,carModel,retired,Pos,stagetimes,overalltimes,positions,penalties
entryId,,,,,,,,,,,
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642063/,Ogier Sébastien - Ingrassia Julien,Ogier Sébastien,Ingrassia Julien,1,Ford Fiesta WRC,False,1.0,"[23:16.6, 14:53.2, 16:36.2, 18:25.3, 8:45.0, 1...","[23:16.6, 38:09.8, 54:46.0, 1:13:11.3, 1:21:56...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642087/,Tänak Ott - Järveoja Martin,Tänak Ott,Järveoja Martin,8,Toyota Yaris WRC,False,2.0,"[23:54.5, 14:57.7, 16:32.3, 18:29.3, 8:42.9, 1...","[23:54.5, 38:52.2, 55:24.5, 1:13:53.8, 1:22:36...","[6, 5, 4, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642089/,Latvala Jari-Matti - Anttila Miikka,Latvala Jari-Matti,Anttila Miikka,7,Toyota Yaris WRC,False,3.0,"[24:05.5, 14:59.7, 16:43.3, 18:39.0, 8:46.1, 1...","[24:05.5, 39:05.2, 55:48.5, 1:14:27.5, 1:23:13...","[7, 6, 6, 5, 5, 5, 5, 5, 3, 3, 3, 3, 3, 3, 3, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642095/,Meeke Kris - Nagle Paul,Meeke Kris,Nagle Paul,10,Citroën C3 WRC,False,4.0,"[25:01.3, 15:21.2, 16:43.9, 18:45.0, 8:50.0, 1...","[25:01.3, 40:22.5, 57:06.4, 1:15:51.4, 1:24:41...","[10, 9, 9, 6, 6, 6, 6, 6, 5, 5, 4, 4, 5, 5, 5,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642081/,Neuville Thierry - Gilsoul Nicolas,Neuville Thierry,Gilsoul Nicolas,5,Hyundai i20 Coupe WRC,False,5.0,"[27:33.4, 14:54.6, 17:08.1, 18:27.2, 8:43.3, 1...","[27:33.4, 42:28.0, 59:36.1, 1:18:03.3, 1:26:46...","[21, 17, 17, 10, 9, 9, 9, 9, 8, 8, 8, 8, 7, 7,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1676582/,Sarrazin Stéphane - Renucci Jacques-Julien,Sarrazin Stéphane,Renucci Jacques-Julien,76,Hyundai i20 R5,True,NaN,"[25:19.8, 16:06.1, 17:16.6, 19:30.5, 9:09.9, 1...","[25:19.8, 41:25.9, 58:42.5, 1:18:13.0, 1:27:22...","[11, 12, 12, 13, 13, 12]","[0, 0, 0, 0, 0, 0]"
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1676575/,Oberti Damien - Escartefigue Thomas,Oberti Damien,Escartefigue Thomas,111,Peugeot 208 R2,True,NaN,"[30:42.5, 19:22.0, 20:11.6, 23:05.5, 11:04.6]","[30:42.5, 50:14.5, 1:10:26.1, 1:33:31.6, 1:44:...","[37, 36, 36, 34, 35]","[0, 0:00, 0, 0, 0]"
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1699357/,Poizot Thibaut - Grand Marion,Poizot Thibaut,Grand Marion,87,Renault Clio RS R3T,True,NaN,"[29:02.1, 18:34.2, 19:46.1, 22:30.1, 10:56.1]","[29:02.1, 47:36.3, 1:07:22.4, 1:30:02.5, 1:40:...","[27, 26, 28, 27, 26]","[0, 0, 0, 0:00, 0]"


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
entryId,,,,,,,,,,,,,,,,,
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642063/,1396.6,2289.8,3286.0,4391.3,4916.3,5928.3,7106.7,7635.4,9169.4,9921.2,11082.9,11742.1,12630.9,13462.3,14108.1,14920.7,15535.5
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642087/,1434.5,2332.2,3324.5,4433.8,4956.7,5962.2,7126.0,7650.3,9247.8,9984.6,11131.0,11781.6,12664.4,13507.3,14154.3,14978.6,15593.8
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642089/,1445.5,2345.2,3348.5,4467.5,4993.6,6006.4,7181.9,7705.5,9268.4,10022.3,11187.3,11841.7,12723.6,13577.4,14219.8,15033.9,15647.5
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642095/,1501.3,2422.5,3426.4,4551.4,5081.4,6095.1,7274.1,7800.9,9417.6,10191.0,11361.5,12026.2,12911.0,13762.2,14401.0,15211.9,15818.6
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642081/,1653.4,2548.0,3576.1,4683.3,5206.6,6218.8,7363.3,7879.5,9504.8,10270.1,11443.1,12091.7,12964.5,13798.4,14432.5,15220.3,15829.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1676582/,1519.8,2485.9,3522.5,4693.0,5242.9,6306.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1676575/,1842.5,3014.5,4226.1,5611.6,6276.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1699357/,1742.1,2856.3,4042.4,5402.5,6058.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
entryId,,,,,,,,,,,,,,,,,
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642063/,1396.6,893.2,996.2,1105.3,525.0,1012.0,1178.4,528.7,1534.0,751.8,1161.7,659.2,888.8,831.4,645.8,812.6,614.8
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642087/,1434.5,897.7,992.3,1109.3,522.9,1005.5,1163.8,524.3,1597.5,736.8,1146.4,650.6,882.8,842.9,647.0,824.3,615.2
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642089/,1445.5,899.7,1003.3,1119.0,526.1,1012.8,1175.5,523.6,1562.9,753.9,1165.0,654.4,881.9,853.8,642.4,814.1,613.6
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642095/,1501.3,921.2,1003.9,1125.0,530.0,1013.7,1179.0,526.8,1616.7,773.4,1170.5,664.7,884.8,851.2,638.8,810.9,606.7
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642081/,1653.4,894.6,1028.1,1107.2,523.3,1012.2,1144.5,516.2,1625.3,765.3,1173.0,648.6,872.8,833.9,634.1,787.8,609.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1676582/,1519.8,966.1,1036.6,1170.5,549.9,1063.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1676575/,1842.5,1162.0,1211.6,1385.5,664.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1699357/,1742.1,1114.2,1186.1,1350.1,656.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
entryId,,,,,,,,,,,,,,,,,
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642063/,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642087/,6,5,4.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642089/,7,6,6.0,5.0,5.0,5.0,5.0,5.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642095/,10,9,9.0,6.0,6.0,6.0,6.0,6.0,5.0,5.0,4.0,4.0,5.0,5.0,5.0,5.0,4.0
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642081/,21,17,17.0,10.0,9.0,9.0,9.0,9.0,8.0,8.0,8.0,8.0,7.0,7.0,7.0,7.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1676582/,11,12,12.0,13.0,13.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1676575/,37,36,36.0,34.0,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1699357/,27,26,28.0,27.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Final Results

Get overall rankings.

In [66]:
final_path = 'https://www.ewrc-results.com/final/{stub}/'

In [67]:
def get_final(stub):
    soup = soupify(final_path.format(stub=stub))
    tables = soup.find_all('table')
    #tables = LH.fromstring(html).xpath('//table')
    df_rally_overall = pd.read_html('<html><body>{}</body></html>'.format(tables[0]))[0]
    #df_rally_overall['badge'] = [img.find('img')['src'] for img in tables[0].findAll("td", {"class": "final-results-icon"}) ]
    df_rally_overall.dropna(how='all', axis=1, inplace=True)
    df_rally_overall.columns=['Pos','CarNum','driverNav','ModelReg','Class', 'Time','GapDiff', 'Speedkm']

    #Get the entry ID - use this as the unique key
    #in column 3, <a title='Entry info and stats'>
    df_rally_overall['entryId']=[a['href'] for a in tables[0].findAll("a", {"title": "Entry info and stats"}) ]
    df_rally_overall.set_index('entryId', inplace=True)

    df_rally_overall[['Driver','CoDriver']] = df_rally_overall['driverNav'].str.extract(r'(?P<Driver>.*)\s+-\s+(?P<CoDriver>.*)')

    df_rally_overall['Historic']= df_rally_overall['Class'].str.contains('Historic')
    df_rally_overall['Class']= df_rally_overall['Class'].str.replace('Historic','')

    df_rally_overall['Pos'] = df_rally_overall['Pos'].astype(str).str.extract(r'(.*)\.')
    df_rally_overall['Pos'] = df_rally_overall['Pos'].astype(int)

    df_rally_overall[['Model','Registration']]=df_rally_overall['ModelReg'].str.extract(r'(?P<Model>.*) \((?P<Registration>.*)\)')

    df_rally_overall["Class Rank"] = df_rally_overall.groupby("Class")["Pos"].rank(method='min')

    return df_rally_overall
    

## Itinerary

In [68]:
itinerary_path = 'https://www.ewrc-results.com/timetable/{stub}/'

In [105]:
def get_itinerary(stub):
    soup = soupify(itinerary_path.format(stub=stub))
    
    event_dist = soup.find('td',text='Event total').parent.find_all('td')[-1].text

    itinerary_df = dfify( soup.find('div', {'class':'timetable'}).find('table') )
    itinerary_df.columns = ['Stage','Name', 'distance', 'Date', 'Time']
    itinerary_df['Leg'] = [nan if 'leg' not in str(x) else str(x).replace('. leg','') for x in itinerary_df['Stage']]
    itinerary_df['Leg'] = itinerary_df['Leg'].fillna(method='ffill')
    itinerary_df['Date'] = itinerary_df['Date'].fillna(method='ffill')

    itinerary_leg_totals = itinerary_df[itinerary_df['Name'].str.contains("Leg total")][['Leg', 'distance']].reset_index(drop=True)

    full_itinerary_df = itinerary_df[~itinerary_df['Name'].str.contains(". leg")]
    full_itinerary_df = full_itinerary_df[~full_itinerary_df['Date'].str.contains(" km")]
    full_itinerary_df = full_itinerary_df.fillna(method='bfill', axis=1)

    #Legs may not be identified but we may want to identify services
    full_itinerary_df['Service'] = [ 'Service' in i for i in full_itinerary_df['distance'] ]
    full_itinerary_df['Service_Num'] = full_itinerary_df['Service'].cumsum()
    full_itinerary_df.reset_index(drop=True, inplace=True)
    #itinerary_df = full_itinerary_df[~full_itinerary_df['Service']].reset_index(drop=True)
    itinerary_df = full_itinerary_df[full_itinerary_df['Stage'].str.startswith('SS')].reset_index(drop=True)
    itinerary_df['Section'] = itinerary_df['Service_Num'].rank(method='dense')
    itinerary_df.drop(columns=['Service', 'Service_Num'], inplace=True)
    
    itinerary_df[['Distance', 'Distance_unit']] = itinerary_df['distance'].str.extract(r'(?P<Distance>[^\s]*)\s+(?P<Distance_unit>.*)?')
    itinerary_df['Distance'] = itinerary_df['Distance'].astype(float)
    itinerary_df.set_index('Stage', inplace=True)

    return event_dist, itinerary_leg_totals, itinerary_df, full_itinerary_df

In [106]:
stub='54762-corbeau-seats-rally-tendring-clacton-2019'
stub='42870-rallye-automobile-de-monte-carlo-2018'
event_dist, itinerary_leg_totals, itinerary_df, full_itinerary_df = get_itinerary(stub)

In [107]:
print(event_dist)
display(itinerary_leg_totals)
display(itinerary_df)
display(full_itinerary_df)

388.59 km


,Leg,distance
0,1,62.18 km
1,2,144.88 km
2,3,117.55 km
3,4,63.98 km


,Name,distance,Date,Time,Leg,Section,Distance,Distance_unit
Stage,,,,,,,,
SS1,Thoard - Sisteron,36.69 km,25. 1.,21:43,1,1.0,36.69,km
SS2,Bayons - Bréziers 1,25.49 km,25. 1.,22:51,1,1.0,25.49,km
SS3,Vitrolles - Oze 1,26.72 km,26. 1.,08:51,2,2.0,26.72,km
SS4,Roussieux - Eygalayes 1,30.54 km,26. 1.,10:04,2,2.0,30.54,km
SS5,Vaumeilh - Claret 1,15.18 km,26. 1.,11:37,2,2.0,15.18,km
SS6,Vitrolles - Oze 2,26.72 km,26. 1.,13:58,2,3.0,26.72,km
SS7,Roussieux - Eygalayes 2,30.54 km,26. 1.,15:11,2,3.0,30.54,km
SS8,Vaumeilh - Claret 2,15.18 km,26. 1.,16:44,2,3.0,15.18,km
SS9,Agnières-en-Dévoluy - Corps 1,29.16 km,27. 1.,08:08,3,4.0,29.16,km


,Stage,Name,distance,Date,Time,Leg,Service,Service_Num
0,Shakedown (Gap),Shakedown (Gap),3.35 km,24. 1.,16:00,NaN,False,0
1,SS1,Thoard - Sisteron,36.69 km,25. 1.,21:43,1,False,0
2,SS2,Bayons - Bréziers 1,25.49 km,25. 1.,22:51,1,False,0
3,Flexi Service A - Gap - 45+3 Min,Flexi Service A - Gap - 45+3 Min,Flexi Service A - Gap - 45+3 Min,25. 1.,23:56,1,True,1
4,Service B - Gap - 15+3 Min,Service B - Gap - 15+3 Min,Service B - Gap - 15+3 Min,26. 1.,08:00,2,True,2
5,SS3,Vitrolles - Oze 1,26.72 km,26. 1.,08:51,2,False,2
6,SS4,Roussieux - Eygalayes 1,30.54 km,26. 1.,10:04,2,False,2
7,SS5,Vaumeilh - Claret 1,15.18 km,26. 1.,11:37,2,False,2
8,Service C - Gap - 30+3 Min,Service C - Gap - 30+3 Min,Service C - Gap - 30+3 Min,26. 1.,12:52,2,True,3
9,SS6,Vitrolles - Oze 2,26.72 km,26. 1.,13:58,2,False,3


In [126]:
_tmp = itinerary_df['Distance']
_tmp.index = [int(i.lstrip('SS')) for i in _tmp.index]
_tmp

1     36.69
2     25.49
3     26.72
4     30.54
5     15.18
6     26.72
7     30.54
8     15.18
9     29.16
10    16.87
11    29.16
12    16.87
13    25.49
14    18.41
15    13.58
16    18.41
17    13.58
Name: Distance, dtype: float64

## Entry List

Get the entry list.

In [74]:
entrylist_path = "https://www.ewrc-results.com/entries/{stub}/"

In [75]:
def get_entry_list(stub):
    entrylist_url = entrylist_path.format(stub=stub)
    
    soup = soupify(entrylist_url)
    entrylist_table = soup.find('div',{'class':'startlist'}).find('table')
    df_entrylist = dfify(entrylist_table)
    
    base_cols = ['CarNum', 'DriverName','CoDriverName','Team','Car','Class']
    for i in range(len(df_entrylist.columns) - len(base_cols)):
        base_cols.append(f'Meta_{i}')
    df_entrylist.columns = base_cols
    df_entrylist['carNum'] = df_entrylist['CarNum'].str.extract(r'#(.*)')
    
    return df_entrylist

In [76]:
get_entry_list(stub)

,CarNum,DriverName,CoDriverName,Team,Car,Class,Meta_0,Meta_1,carNum
0,NaN,Henry Patrick,Lombard Magali,Renault Mégane IV RS,NaN,NaN,NaN,Course car,NaN
1,NaN,Saby Bruno,Mondon Cédric,Renault Mégane IV RS,NaN,NaN,NaN,Course car,NaN
2,NaN,Sainz Jr Carlos,Sanjuan De Eusebio Rodrigo,Renault Mégane IV RS,NaN,NaN,NaN,Course car,NaN
3,#1,Ogier Sébastien,Ingrassia Julien,Ford Fiesta WRCM-Sport Ford WRT,[S1 FMC],RC1,M,NaN,1
4,#2,Evans Elfyn,Barritt Daniel,Ford Fiesta WRCM-Sport Ford WRT,[X1 FMC],RC1,M,NaN,2
...,...,...,...,...,...,...,...,...,...
65,#115,Dessi Marc,Dessi Vanessa,Renault Twingo RS R2,NaN,RC4,NaN,NaN,115
66,#116,Martini Eric,Pengial Guy,Renault Twingo RS R1,NaN,RC5,NaN,NaN,116
67,#117,Mari Alain,Cesari Jean-Dominique,Renault Twingo RS R1,NaN,RC5,NaN,NaN,117
68,#118,Desbordes Richard,Fustier Jean-Luc,Citroën DS3 R1,NaN,RC5,NaN,NaN,118


## Rebasers

Utils for rebasing

In [85]:
def _rebaseTimes(times, bib=None, basetimes=None):
    ''' Rebase times relative to specified driver. '''
    #Should we rebase against entryId, so need to lool that up. In which case, leave index as entryId
    if bib is None and basetimes is None: return times
    #bibid = codes[codes['Code']==bib].index.tolist()[0]
    if bib is not None:
        return times - times.loc[bib]
    if times is not None:
        return times - basetimes
    return times

## `EWRC` Class
Create a class to that can be used to gran all the results for a particular rally, as required.

We fudge the definition of class functions so that we can separately define and functions in a standalione way. This is probably *not good practice*...!

In [102]:
class EWRC:
    """Class for eWRC data for a particular rally."""

    def __init__(self, stub):
        self.stub = stub
        
        self.stage_result_links = None
        
        self.df_rally_overall = None
        
        self.df_allInOne = None #we don't actually use this?
        self.df_overall = None
        self.df_stages = None
        self.df_overall_pos = None
        self.df_stages_pos = None
        
        self.df_overall_rebased_to_leader = None
        self.df_stages_rebased_to_overall_leader = None
        self.df_stages_rebased_to_stage_winner = None
        
        self.event_dist = None
        self.df_itinerary_leg_totals = None
        self.df_itinerary = None
        self.df_full_itinerary =None
        self.stage_distances = None
        
        self.df_entry_list = None
 
        self.df_stage_result = pd.DataFrame(columns=stage_result_cols)
        self.df_stage_overall = pd.DataFrame(columns=stage_overall_cols)
        self.df_stage_retirements = pd.DataFrame(columns=retirement_cols+retirement_extra_cols)
        self.df_stage_penalties = pd.DataFrame(columns=penalty_cols+penalty_extra_cols)

    def set_rebased_times(self):
        if self.df_stages_rebased_to_overall_leader is None \
                or self.df_stages_rebased_to_stage_winner is None \
                or self.df_stages_rebased_to_stage_winner is None:
            #print('setting rebased times...')
            self.get_stage_times()
            leaderStagetimes = self.df_stages.iloc[0]
            self.df_stages_rebased_to_overall_leader = self.df_stages.apply(_rebaseTimes, basetimes=leaderStagetimes, axis=1)
            #Now rebase to the stage winner
            self.df_stages_rebased_to_stage_winner = self.df_stages_rebased_to_overall_leader.apply(_rebaseTimes, basetimes=self.df_stages_rebased_to_overall_leader.min(), axis=1)

            leaderTimes = self.df_overall.min()
            self.df_overall_rebased_to_leader = self.df_overall.apply(_rebaseTimes, basetimes=leaderTimes, axis=1)

    
    def get_final(self):
        if self.df_rally_overall is None:
            self.df_rally_overall = get_final(self.stub)
        return self.df_rally_overall
        
    def get_stage_times(self):
        if self.df_overall is None or self.df_stages is None or self.df_overall_pos is None:
            self.df_allInOne, self.df_overall, self.df_stages, \
                self.df_overall_pos, self.df_stages_pos = get_stage_times(self.stub)

        return self.df_allInOne, self.df_overall, self.df_stages, self.df_overall_pos
    
    def get_itinerary(self):
        if self.event_dist is None or self.df_itinerary_leg_totals is None \
            or self.df_itinerary is None or self.df_full_itinerary is None:
                self.event_dist, self.df_itinerary_leg_totals, \
                    self.df_itinerary, self.df_full_itinerary_df = get_itinerary(self.stub)
        
        _stage_distances = self.df_itinerary['Distance']
        _stage_distances.index = [int(i.lstrip('SS')) for i in _stage_distances.index]
        self.stage_distances = _stage_distances
        return self.event_dist, self.df_itinerary_leg_totals, \
                self.df_itinerary, self.df_full_itinerary
    
    def get_entry_list(self):
        if self.df_entry_list is None:
            self.df_entry_list = get_entry_list(self.stub)
        return self.df_entry_list
    
    def get_stage_result_links(self):
        if self.stage_result_links is None:
            self.stage_result_links = get_stage_result_links(self.stub)
        return self.stage_result_links
    
    def get_stage_results(self, stage=None):
        #for now, just return what we have with stage as None
        if stage is None:
            return self.df_stage_result, self.df_stage_overall, \
                    self.df_stage_retirements, self.df_stage_penalties
        # Could maybe change that to get everything?
        stages = stage if isinstance(stage,list) else [stage]
        if stages:
            links = self.get_stage_result_links()
            if 'all' in stages:
                stages = [k for k in links.keys() if 'leg' not in k]
            elif 'final' in stages or 'last' in stages:
                stages = [k for k in links.keys() if 'leg' not in k][-1]
            for stage in stages:
                if stage not in self.df_stage_result['Stage'].unique() and stage in links:
                    df_stage_result, df_stage_overall, df_stage_retirements, \
                        df_stage_penalties = get_stage_results(links[stage])
                    self.df_stage_result = self.df_stage_result.append(df_stage_result, sort=False).reset_index(drop=True)
                    self.df_stage_overall = self.df_stage_overall.append(df_stage_overall, sort=False).reset_index(drop=True)
                    self.df_stage_retirements = self.df_stage_retirements.append(df_stage_retirements, sort=False).reset_index(drop=True)
                    self.df_stage_penalties = self.df_stage_penalties.append(df_stage_penalties, sort=False).reset_index(drop=True)
        
        if stages:
            return self.df_stage_result[self.df_stage_result['Stage'].isin(stages)], \
                    self.df_stage_overall[self.df_stage_overall['Stage'].isin(stages)], \
                    self.df_stage_retirements[self.df_stage_retirements['Stage'].isin(stages)], \
                    self.df_stage_penalties[self.df_stage_penalties['Stage'].isin(stages)]
        
        return self.df_stage_result, self.df_stage_overall, \
                self.df_stage_retirements, self.df_stage_penalties

In [103]:
ewrc=EWRC(rally_stub)

In [88]:
#df_stage_result, df_stage_overall, df_stage_retirements, df_stage_penalties
ewrc.get_stage_results()

(Empty DataFrame
 Columns: [Pos, CarNum, Desc, Class, Time, GapDiff, Speedkm, Stage, StageName, StageDist, Gap, Diff, Speed, Dist, entryId, model, navigator, PosNum]
 Index: [], Empty DataFrame
 Columns: [PosChange, CarNum, Desc, Class, Time, GapDiff, Speedkm, Stage, StageName, StageDist, Pos, Change, Gap, Diff, Speed, Dist]
 Index: [], Empty DataFrame
 Columns: [CarNum, driverNav, Model, Status, Driver, CoDriver, Stage]
 Index: [], Empty DataFrame
 Columns: [CarNum, driverNav, Model, PenReason, Driver, CoDriver, Stage, Time, Reason]
 Index: [])

In [94]:
ewrc.get_stage_results('all')

(    Pos CarNum                                               Desc Class  \
 0     1      1      Ogier Sébastien - Ingrassia J.Ford Fiesta WRC  RC1M   
 1     2      4  Mikkelsen Andreas - Jæger A.Hyundai i20 Coupe WRC  RC1M   
 2     3      9        Lappi Esapekka - Ferm JanneToyota Yaris WRC  RC1M   
 3     4      6  Sordo Dani - del Barrio CarlosHyundai i20 Coup...  RC1M   
 4     5     11           Breen Craig - Martin ScottCitroën C3 WRC  RC1M   
 ..   ..    ...                                                ...   ...   
 964  43     96      Covi Carlo - Ometto Pietro EliaPeugeot 208 R2   RC4   
 965  44    118       Desbordes Richard - Fustier J.Citroën DS3 R1   RC5   
 966  45    119  Berard Christophe - Bernabo C.Renault Twingo R...   RC5   
 967  46    116     Martini Eric - Pengial GuyRenault Twingo RS R1   RC5   
 968  47    109     Villy Jonathan - Villy RonaldCitroën C2 R2 Max   RC4   
 
         Time         GapDiff   Speedkm Stage  \
 0    23:16.6             NaN      94

In [104]:
ewrc.set_rebased_times()
display(ewrc.df_stages_rebased_to_overall_leader)

setting rebased times...


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
entryId,,,,,,,,,,,,,,,,,
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642063/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642087/,37.9,4.5,-3.9,4.0,-2.1,-6.5,-14.6,-4.4,63.5,-15.0,-15.3,-8.6,-6.0,11.5,1.2,11.7,0.4
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642089/,48.9,6.5,7.1,13.7,1.1,0.8,-2.9,-5.1,28.9,2.1,3.3,-4.8,-6.9,22.4,-3.4,1.5,-1.2
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642095/,104.7,28.0,7.7,19.7,5.0,1.7,0.6,-1.9,82.7,21.6,8.8,5.5,-4.0,19.8,-7.0,-1.7,-8.1
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1642081/,256.8,1.4,31.9,1.9,-1.7,0.2,-33.9,-12.5,91.3,13.5,11.3,-10.6,-16.0,2.5,-11.7,-24.8,-5.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1676582/,123.2,72.9,40.4,65.2,24.9,51.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1676575/,445.9,268.8,215.4,280.2,139.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/entryinfo/42870-rallye-automobile-de-monte-carlo-2018/1699357/,345.5,221.0,189.9,244.8,131.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
ewrc.get_stage_result_links()

In [ ]:
ewrc.get_stage_times()

In [ ]:
ewrc.get_itinerary()

In [ ]:
ewrc.get_entry_list()

In [ ]:
ewrc=EWRC('54464-corsica-linea-tour-de-corse-2019')

In [ ]:
ewrc.get_entry_list()

In [ ]:
ewrc.get_stage_results('SS2')